### environment setup

In [ ]:
# !conda install -c anaconda tk -y
# !conda install -c anaconda pillow -y
!conda install -c conda-forge hdf5storage -y

### image loader

In [ ]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision
from torch import nn
import PIL.Image
from scipy.io import savemat, loadmat
from datetime import datetime
# from visdom import Tensor, Visdom
import glob
import pickle
import random
import sys
from sklearn.decomposition import PCA
import copy

# Define data loading step
class image_transform_dataloader(Dataset):

    def __init__(self, root_dir, start_index, weather_num, town_num, run_sample_size, sample_step, device=None):
        self.root_dir = root_dir

        self.weather_num = weather_num
        self.town_num = town_num
        self.run_sample_size = run_sample_size
        self.sample_step = sample_step
        self.start_index = start_index
        self.device = device
        
        self.towns_dict = {
            0: "Town01",
            1: "Town02",
            2: "Town03",
            3: "Town04",
            4: "Town05",
            5: "Town06",
            6: "Town07",
            7: "Town08",
            }

        self.weather_dict = {
            0: "ClearNight",
            1: "WetSunset",
            2: "WetCloudyNoon",
            3: "ClearNoon",
            4: "ClearSunset",
            5: "HardRainSunset",
            6: "FogMorning",
            }

        print("---Summary---")
        print("num of towns:", self.town_num)
        print("num of weather in each town:", self.weather_num)
        print("sample size of each town:", self.run_sample_size)
        print("sample step:", self.sample_step)

        self.num_images = int(self.run_sample_size/self.sample_step)
        print("num of total images per weather:", self.num_images)
        print("-----------")

        return

    def rgb_images_index_for_labeling(self, i_town, i_weather):

        self.rgb_images = np.zeros((self.num_images, 3, 900, 1600))

        for index in range(self.num_images):
            
            # calc index
            index_in_town_sample = index*self.sample_step

            # folder
            self.rgb_image_folder_str = self.towns_dict[i_town] + '_' + self.weather_dict[i_weather]

            # index image
            current_rgb_str = 'rgb_{idx:06}.jpg'
            self.current_wildcard_rgb_name = os.path.join(self.root_dir, self.rgb_image_folder_str, \
                    current_rgb_str.format(idx = index_in_town_sample + self.start_index))

            current_rbg_name = glob.glob(self.current_wildcard_rgb_name)[0]

            rgb_image = PIL.Image.open(current_rbg_name).convert('RGB') #HxWxC
            rgb_image = np.transpose(np.asarray(rgb_image), (2, 0, 1)) #HxWxC into CxHxW format
                        
            self.rgb_images[index] = rgb_image
            
        return self.rgb_images



In [ ]:
## get weight of each component from TR images
# each run 5*60=300s, TR=1.5s, 50 fps,
# num of TR 300/1.5=200TR,
# start index of image = 60, 
# sample step = TR*50 = 75,
# sample size for each town 300*50 = 15000

root_dir="/media/statespace/S/recording/output_synchronized"

num_TR = 200
run_sample_size = 15000
town_num = 8
weather_num = 7
sample_step = 75
num_images = int(run_sample_size/sample_step)

data_sampler = image_transform_dataloader(root_dir=root_dir, start_index=60, town_num=town_num, weather_num=weather_num,run_sample_size=run_sample_size, sample_step=sample_step)


In [ ]:
rgb_images_array = np.zeros((town_num, num_TR, 3, 900, 1600))

for i_town in range(town_num):
  print("i_town:",i_town)
  rgb_images = data_sampler.rgb_images_index_for_labeling(i_town=i_town, i_weather=3)
  rgb_images_array[i_town] = rgb_images

print('(num_towns, num_images, C, H, W)=', rgb_images_array.shape)


#### save and load

In [ ]:
# save
import hdf5storage

matdic = {"rgb_images_array":rgb_images_array}
save_dir = "/media/statespace/Spatial/sptialworkspace/spatialfMRI/simulation/carla_ws/label_road/"
hdf5storage.savemat(save_dir+"rgb_images_array.mat", matdic)


In [ ]:
# load
import hdf5storage

save_dir = "/media/statespace/Spatial/sptialworkspace/spatialfMRI/simulation/carla_ws/label_road/"
matdic = hdf5storage.loadmat(save_dir+"rgb_images_array.mat")
rgb_images_array = matdic["rgb_images_array"]
print('(num_towns, num_images, C, H, W)', rgb_images_array.shape)


### tkinter 

#### load images

In [1]:
import hdf5storage

save_dir = "/media/statespace/Spatial/sptialworkspace/spatialfMRI/simulation/carla_ws/label_road/"
matdic = hdf5storage.loadmat(save_dir+"rgb_images_array.mat")
rgb_images_array = matdic["rgb_images_array"]
print('(num_towns, num_images, C, H, W)', rgb_images_array.shape)

# i_town is from 0 to 7 (all 8 towns)
i_town = 0
rgb_images = rgb_images_array[i_town]

(num_towns, num_images, C, H, W) (8, 200, 3, 900, 1600)


#### label gui launch

In [3]:
# importing required packages
import tkinter
from PIL import ImageTk, Image
import os
import numpy as np
import sys

# creating main window
window = tkinter.Tk()

# place windown on the center
screen_width = window.winfo_screenwidth()  # Width of the screen
screen_height = window.winfo_screenheight() # Height of the screen

width = screen_width*0.6 # Width 
height = screen_height*0.8 # Height

x = (screen_width/2) - (width/2)
y = (screen_height/2) - (height/2)

window.geometry('%dx%d+%d+%d' % (width, height, x, y))
window.title('Labeling Road Structure')

# opens the image
global i_rgb_image
i_rgb_image = 0
# imagetk read first image
temp_rgb_image = rgb_images[i_rgb_image] #CxHxW format
temp_rgb_image = np.transpose(temp_rgb_image, (1, 2, 0)) #HxWxC
pil_temp_rgb_image = Image.fromarray((temp_rgb_image).astype(np.uint8))

# imagetk
img_town_temp = pil_temp_rgb_image.resize((int(screen_width*0.6), int(screen_height*0.5)), Image.Resampling.LANCZOS)
img_town_cam = ImageTk.PhotoImage(img_town_temp)
# i_rgb_image = i_rgb_image + 1

# read any image
# img_town_cam = Image.open("rgb_town07.jpg")
# img_town_cam = img_town_cam.resize((int(screen_width*0.6), int(screen_height*0.5)), Image.Resampling.LANCZOS)
# img_town_cam = ImageTk.PhotoImage(img_town_cam)

# create canvas
canvas_town_cam = tkinter.Canvas(window, width=int(screen_width*0.6), height=int(screen_height*0.5))
img_town_container = canvas_town_cam.create_image(0,0, anchor='nw', image=img_town_cam)
canvas_town_cam.pack(side='top')

# opens the image
img_road_label = Image.open("roadtypes.png")
img_road_label = img_road_label.resize((int(screen_width*0.6), int(screen_height*0.2)), Image.Resampling.LANCZOS)
img_road_label = ImageTk.PhotoImage(img_road_label)

# create canvas
canvas_road_label = tkinter.Canvas(window, width=int(screen_width*0.6), height=int(screen_height*0.2))
img_road_label_container = canvas_road_label.create_image(0, 0, anchor='nw', image=img_road_label)
canvas_road_label.pack(side='top')

# # user information
# tkinter.Label(window, text='Types:', font=('Helvetica bold', 40)).place(x=0, y=750)
rb_rely = 0.91
rb_height = int(screen_height*0.0024)
v = tkinter.StringVar(window, "1")
variable_radio = tkinter.IntVar()

tkinter.Radiobutton(window, text = "Straight", variable = variable_radio, font=('Helvetica bold', 16),
            value = "1", indicator = 0, height=rb_height, width=int(screen_width*0.006),
            background = "white").place(relx = 0.05, rely = rb_rely, anchor = tkinter.CENTER)

tkinter.Radiobutton(window, text = "Left or right turn", variable = variable_radio, font=('Helvetica bold', 16),
            value = "2", indicator = 0, height=rb_height, width=int(screen_width*0.013),
            background = "white").place(relx = 0.23, rely = rb_rely, anchor = tkinter.CENTER)

tkinter.Radiobutton(window, text = "T junction", variable = variable_radio, font=('Helvetica bold', 16),
            value = "3", indicator = 0, height=rb_height, width=int(screen_width*0.013),
            background = "white").place(relx = 0.48, rely = rb_rely, anchor = tkinter.CENTER)

tkinter.Radiobutton(window, text = "Crossroads", variable = variable_radio, font=('Helvetica bold', 16),
            value = "4", indicator = 0, height=rb_height, width=int(screen_width*0.010),
            background = "white").place(relx = 0.71, rely = rb_rely, anchor = tkinter.CENTER)

tkinter.Radiobutton(window, text = "Others", variable = variable_radio, font=('Helvetica bold', 16),
            value = "5", indicator = 0, height=rb_height, width=int(screen_width*0.010),
            background = "white").place(relx = 0.89, rely = rb_rely, anchor = tkinter.CENTER)

# scale
variable_scale = tkinter.DoubleVar()
s2 = tkinter.Scale(window, variable = variable_scale,
           from_ = 1, to = 10, length=int(screen_width*0.15),
           orient = tkinter.HORIZONTAL).place(relx = 0.3, rely = 0.96, anchor = tkinter.CENTER)

lalel_scale = tkinter.Label(window, text = "Degree of belief:").place(relx = 0.1, rely = 0.97, anchor = tkinter.CENTER)

road_structure_list = ['straight', 'Left or right turn', 'T junction', 'Crossroads', 'Others']

road_labels_array = np.zeros((len(road_structure_list), rgb_images.shape[0]))

# process func
def labeling_road_func():

    global i_rgb_image

    print("--------------------------")
    print("image index:", i_rgb_image)
    # print("radio variable:", variable_radio.get())
    print("road:", road_structure_list[variable_radio.get()-1])
    print("belief:", int(variable_scale.get()))

    road_labels_array[variable_radio.get()-1][i_rgb_image] = int(variable_scale.get())

    # for the next image index
    i_rgb_image = i_rgb_image + 1

    if i_rgb_image == rgb_images.shape[0]:
        print('Finish town-0'+str(i_town+1)+' labeling.')
        window.destroy()
        sys.exit()

    # read new image
    temp_rgb_image = rgb_images[i_rgb_image] #CxHxW format
    temp_rgb_image = np.transpose(temp_rgb_image, (1, 2, 0)) #HxWxC
    pil_temp_rgb_image = Image.fromarray((temp_rgb_image).astype(np.uint8))

    # imagetk
    img_town_temp = pil_temp_rgb_image.resize((int(screen_width*0.6), int(screen_height*0.5)), Image.Resampling.LANCZOS)
    img_town_temp = ImageTk.PhotoImage(img_town_temp)
    
    # update
    canvas_town_cam.itemconfig(img_town_container,image=img_town_temp)
    canvas_town_cam.imgref = img_town_temp


# next image
btn = tkinter.Button(window, text = 'Next image', command = labeling_road_func).place(relx = 0.5, rely = 0.97, anchor = tkinter.CENTER)

# running the application
window.mainloop()


--------------------------
image index: 0
road: Others
belief: 1
--------------------------
image index: 1
road: Others
belief: 1
--------------------------
image index: 2
road: Others
belief: 1
--------------------------
image index: 3
road: Others
belief: 1
--------------------------
image index: 4
road: Others
belief: 1
--------------------------
image index: 5
road: Others
belief: 1
--------------------------
image index: 6
road: Others
belief: 1
--------------------------
image index: 7
road: Others
belief: 1
--------------------------
image index: 8
road: Others
belief: 1
--------------------------
image index: 9
road: Others
belief: 1
--------------------------
image index: 10
road: Others
belief: 1
--------------------------
image index: 11
road: Others
belief: 1
--------------------------
image index: 12
road: Others
belief: 1
--------------------------
image index: 13
road: Others
belief: 1
--------------------------
image index: 14
road: Others
belief: 1
--------------------

SystemExit: 0

/home/statespace/anaconda3/envs/deeplearning/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


#### save

In [ ]:
from scipy.io import savemat

matdic = {"road_labels_array":road_labels_array}
save_dir = "/media/statespace/Spatial/sptialworkspace/spatialfMRI/simulation/carla_ws/label_road/"
savemat(save_dir+"road_labels_town_"+str(i_town)+".mat", matdic)
